# Analyze Energy Ratios with Wake Steering

In this notebook, we will compare energy ratios computed using data from baseline and wake steering control periods. The energy ratios for the two control cases will be compared to evaluate the impact of wake steering on the energy production of the test turbines.

In [ ]:
import copy
from pathlib import Path

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from flasc.energy_ratio import energy_ratio as er, total_uplift as tup
from flasc.energy_ratio.energy_ratio_input import EnergyRatioInput
from flasc.utilities_examples import load_floris_smarteole as load_floris
from flasc.visualization import plot_binned_mean_and_ci

In [ ]:
# Suppress warnings
import warnings

warnings.filterwarnings("ignore")

# Step 0: Load processed data

Load the processed SCADA data with power curve filtering and northing calibration applied and inspect

In [ ]:
def load_data():
    root_path = Path.cwd()
    f = root_path / "postprocessed" / "df_scada_data_60s_filtered_and_northing_calibrated.ftr"
    df_scada = pd.read_feather(f)

    # # Optionally: downsample to [x] minute averages to speed up things
    # cols_angular = [c for c in df_scada if (("wd_" in c) or ("yaw_" in c))]
    # df_scada = fto.df_downsample(
    #     df_scada,
    #     cols_angular=cols_angular,
    #     window_width=td(seconds=600),
    # )

    return df_scada


df_scada = load_data()

In [ ]:
df_scada.describe()

In [ ]:
df_scada.columns

# Step 1: Evaluate offsets

Compare the targeted offsets against what was achieved (as measured by the vane on the steering turbine SMV6 (index=5))

In [ ]:
# Parameters
start_of_offset = 200  # deg
end_of_offset = 240  # deg

In [ ]:
# Add a rounded ws column
df_scada = df_scada.assign(ws_round=df_scada.ws_005.round(), wd_to_plot=df_scada.wd_005)

# Limit to a few wind speeds for plotting
wind_speeds_to_plot = [6, 8, 10, 12]
df_plot = df_scada[df_scada.ws_round.isin(wind_speeds_to_plot)]

# Melt together targeted and achieved
df_plot = (
    df_plot[["wd_to_plot", "ws_round", "target_yaw_offset_005", "wind_vane_005", "control_mode"]]
    # .melt(id_vars=['wd_round','ws_round','control_mode'],
    #       var_name='offset_type',
    #       value_name='offset_value')
    .sort_values(["wd_to_plot", "ws_round"])
)

# Set up binning plots
x_edges = np.arange(start_of_offset - 20, end_of_offset + 20, 2)

fig, axarr = plt.subplots(len(wind_speeds_to_plot), 2, figsize=(10, 10))

for ws_idx, ws in enumerate(wind_speeds_to_plot):
    for c_idx, control_mode in enumerate(["baseline", "controlled"]):
        ax = axarr[ws_idx, c_idx]
        df_sub = df_plot[(df_plot.ws_round == ws) & (df_plot.control_mode == control_mode)]

        plot_binned_mean_and_ci(
            df_sub.wd_to_plot,
            df_sub.target_yaw_offset_005,
            color="k",
            x_edges=x_edges,
            label="Targetted Offset",
            ax=ax,
        )

        plot_binned_mean_and_ci(
            df_sub.wd_to_plot,
            df_sub.wind_vane_005,
            color="r",
            x_edges=x_edges,
            label="SMV6 Vane Measurement",
            ax=ax,
        )

        ax.set_xlim([start_of_offset - 20, end_of_offset + 20])
        ax.set_ylim([-30, 50])
        ax.grid(True)
        ax.axvline(start_of_offset, color="k", ls="--")
        ax.axvline(end_of_offset, color="k", ls="--")

        if ws_idx == 0:
            ax.set_title(f"Control Mode: {control_mode}")

        if c_idx == 0:
            ax.set_ylabel(f"Wind Speed: {ws} m/s\nYaw Misalignment (deg)")
        else:
            ax.set_ylabel("Yaw Misalignment (deg)")

        if (ws_idx == 0) and (c_idx == 0):
            ax.legend()

In [ ]:
# Limit the data to this region
df_scada = df_scada[
    (df_scada.wd_smarteole > (start_of_offset - 20))
    & (df_scada.wd_smarteole < (end_of_offset + 20))
]

# Step 2: Prepare the data for computing energy ratios

The energy ratio class as presently implemented requires explicit identification of the dataframe of the reference wind direction, wind speed, and power columns: "wd," "ws," and "pow_ref." Here, we will set these equal to the reference variables used in the SMARTEOLE wake steering experiment, which we computed in "02_download_and_format_dataset.ipynb".

In [ ]:
# Assign wd, ws and pow ref
# Subset dataframe
df_scada = df_scada.assign(
    wd=lambda df_: df_["wd_smarteole"],
    ws=lambda df_: df_["ws_smarteole"],
    pow_ref=lambda df_: df_["pow_ref_smarteole"],
)

In [ ]:
# Split df_scada into baseline and wake steeering on "controlled"
# periods
df_base = df_scada[df_scada.control_mode == "baseline"]
df_con = df_scada[df_scada.control_mode == "controlled"]

In [ ]:
# Construct energy ratio input object using default 10 blocks
# TO DO: allow color specification in EnergyRatioInput
er_in = EnergyRatioInput([df_base, df_con], ["Baseline", "Controlled"])
er_colors = {"Baseline": "black", "Controlled": "red"}

# Step 3: Energy ratio plots of SCADA data

Plot the energy ratio comparisons for the downstream turbine (SMV5 (Turbine 004)), upstream turbine (SMV6 (Turbine 005)) and the two turbines combined against the reference power (established in earlier notebooks).

In contrast to the energy ratios computed for baseline control only, the energy ratios for the "Baseline" and "Controlled" cases are automatically rebalanced to help ensure a fair comparison between the energy produced with the two control modes. Rebalancing involves weighting the contribution of the energy produced in each wind speed bin evenly for the two cases, even if the data collected for the two scenarios represent different wind speed distributions.

In [ ]:
# Check energy ratios on SMV5 (index=4) (downstream turbine)
er_out = er.compute_energy_ratio(
    er_in,
    test_turbines=[4],
    use_predefined_ref=True,
    use_predefined_wd=True,
    use_predefined_ws=True,
    wd_step=2.0,
    ws_step=1.0,
)
ax = er_out.plot_energy_ratios(color_dict=er_colors)
ax[0].set_title("Downstream Turbine: SMV5 (004)")

In [ ]:
# Check energy ratios on SMV6 (index=5) (upstream turbine)
er_out = er.compute_energy_ratio(
    er_in,
    test_turbines=[5],
    use_predefined_ref=True,
    use_predefined_wd=True,
    use_predefined_ws=True,
    wd_step=2.0,
    ws_step=1.0,
)
ax = er_out.plot_energy_ratios(color_dict=er_colors)
ax[0].set_title("Upstream Turbine: SMV6 (005)")

In [ ]:
# Check energy ratios on for combination of SMV5 and SMV6
er_out = er.compute_energy_ratio(
    er_in,
    test_turbines=[5, 4],
    use_predefined_ref=True,
    use_predefined_wd=True,
    use_predefined_ws=True,
    wd_step=2.0,
    ws_step=1.0,
)
ax = er_out.plot_energy_ratios(color_dict=er_colors)
ax[0].set_title("Upstream and Downstream Turbines Combined: SMV5+SMV6 (004+005)")

# Step 4: Use bootstrapping to estimate uncertainty

Now, we'll plot the energy ratio comparisons for the upstream and downstream turbines combined with bootstrapping to quantify uncertainty.

First we'll use normal bootstrapping, in which the timestamps used in the energy ratio calculation are randomly resampled with replacement each bootstrap iteration.

The shaded regions in the plots below indicate the 95% confidence intervals of the mean energy ratio estimates.

In [ ]:
# Check energy ratios on for combination of SMV5 and SMV6
# Using N=40 bootstraps
er_out = er.compute_energy_ratio(
    er_in,
    test_turbines=[5, 4],
    use_predefined_ref=True,
    use_predefined_wd=True,
    use_predefined_ws=True,
    wd_step=2.0,
    ws_step=1.0,
    N=40,
)
er_out.plot_energy_ratios(color_dict=er_colors)
# s.plot_energy_ratios(show_barplot_legend=False)

As expected, the uncertainty is higher for wind directions with fewer data samples.

Next we'll use block bootstrapping, in which the data used in the energy ratio calculation are first grouped into blocks of consecutive timestamps and the individual blocks are randomly resampled with replacement each bootstrap iteration. In this example, we'll divide the data into 10 blocks. Block bootstrapping can provide a more realistic estimate of the true uncertainty than normal bootstrapping.

In [ ]:
# Check energy ratios on for combination of SMV5 and SMV6
# Using N=40 bootstraps without block bootstrapping
er_in_noblocks = EnergyRatioInput(
    [df_base, df_con], ["Baseline", "Controlled"], num_blocks=min(len(df_base), len(df_con))
)
er_out = er.compute_energy_ratio(
    er_in_noblocks,
    test_turbines=[5, 4],
    use_predefined_ref=True,
    use_predefined_wd=True,
    use_predefined_ws=True,
    wd_step=2.0,
    ws_step=1.0,
    N=40,
)
er_out.plot_energy_ratios(color_dict=er_colors)

# Step 6: Plot the energy ratio gain with bootstrapping

Next, we will plot the difference between the energy ratios with and without wake steering. These plots will show how much wake steering increases (or decreases) energy production as a function of wind direction. 

The first plot shows the energy ratio gain for the downstream turbine (SMV5) and the second plot shows the energy gain for the two turbines combined.

In [ ]:
# SMV5 (Downstream)
er_out = er.compute_energy_ratio(
    er_in,
    test_turbines=[4],
    use_predefined_ref=True,
    use_predefined_wd=True,
    use_predefined_ws=True,
    wd_step=2.0,
    ws_step=1.0,
    uplift_pairs=[("Baseline", "Controlled")],
    uplift_names=["Uplift"],
    N=40,
)
ax = er_out.plot_uplift(color_dict={"Uplift": "black"})
ax[0].set_title("Downstream Turbine: SMV5 (004)")

In [ ]:
# SMV5 + SMV6 (Combined)
er_out = er.compute_energy_ratio(
    er_in,
    test_turbines=[4, 5],
    use_predefined_ref=True,
    use_predefined_wd=True,
    use_predefined_ws=True,
    wd_step=2.0,
    ws_step=1.0,
    uplift_pairs=[("Baseline", "Controlled")],
    uplift_names=["Uplift"],
    N=40,
)
ax = er_out.plot_uplift(color_dict={"Uplift": "black"})
ax[0].set_title("Upstream and Downstream Turbines Combined: SMV5+SMV6 (004+005)")

The above plots indicate that wake steering increases the energy of the downstream turbine significantly for wind directions between 205 and 220 degrees. When including the upstream turbine, the impact on energy is smaller, but wake steering still yields an increase in energy.

# Step 7: Compare with FLORIS

Now we will compare the observed energy ratios and energy ratio gains for baseline and wake steering control based on SCADA data to the equivalent FLORIS predictions.

In [ ]:
# Define a small function to build a dataframe of FLORIS predictions with yaw misalignment
# for comparison with SCADA

num_turbines = 7


def build_floris_data_frame(
    fi_in, wind_speeds, wind_directions, yaw_angles_in=None, ref_pow_columns=[0, 1, 2, 6]
):
    fi_ = copy.deepcopy(fi_in)

    yaw_angles = np.zeros([len(wind_directions), 1, num_turbines])
    if yaw_angles_in is not None:
        yaw_angles[:, 0, 5] = yaw_angles_in  # Apply angles to SMV6

    fi_.reinitialize(wind_speeds=wind_speeds, wind_directions=wind_directions, time_series=True)

    fi_.calculate_wake(yaw_angles=yaw_angles)

    turbine_powers = fi_.get_turbine_powers().squeeze() / 1000.0

    df_ = pd.DataFrame(
        turbine_powers, columns=["pow_{:03d}".format(ii) for ii in range(num_turbines)]
    )

    df_ = df_.assign(
        ws=wind_speeds,
        wd=wind_directions,
        pow_ref=df_[["pow_{:03d}".format(ii) for ii in ref_pow_columns]].mean(axis=1),
    )

    return df_

## Comparison assuming no wind direction uncertainty/variability in FLORIS

Compute FLORIS predictions of power production for the time series of reference wind directions and wind speeds in the SCADA data set, using the Gauss-Curl Hybrid wake model. The yaw misalignment of the steered turbine, SMV6 (005), is set to the measured wind vane angle at each timestamp. Yaw misalignments for all other turbines are set to 0.

In [ ]:
fi, _ = load_floris("gch", wd_std=0.0)

In [ ]:
# Resimulate FLORIS using time-domain sim assuming all
# yaws are 0 except for SMV5, which follows either target exactly or
# what is measured via the vane

# Baseline / Perfect yawing
wind_speeds_baseline = df_base.ws.values
wind_directions_baseline = df_base.wd.values
yaw_angles_baseline_target = None
yaw_angles_baseline_measured = df_base.wind_vane_005.values

wind_speeds_con = df_con.ws.values
wind_directions_con = df_con.wd.values
yaw_angles_con_target = df_con.target_yaw_offset_005.values
yaw_angles_con_measured = df_con.wind_vane_005.values

# Compute FLORIS assuming target offsets and no wd std
df_floris_target_offset_baseline = build_floris_data_frame(
    fi, wind_speeds_baseline, wind_directions_baseline, yaw_angles_baseline_target
)
df_floris_target_offset_con = build_floris_data_frame(
    fi, wind_speeds_con, wind_directions_con, yaw_angles_con_measured
)

Now we'll plot a comparison of the energy ratios based on SCADA data and FLORIS predictions for the downstream turbine SMV5 (004).

In [ ]:
# Construct energy ratio object
er_in = EnergyRatioInput(
    [df_base, df_con, df_floris_target_offset_baseline, df_floris_target_offset_con],
    ["Baseline [SCADA]", "Controlled [SCADA]", "Baseline [FLORIS]", "Controlled [FLORIS]"],
)

er_out = er.compute_energy_ratio(
    er_in,
    test_turbines=[4],
    use_predefined_ref=True,
    use_predefined_wd=True,
    use_predefined_ws=True,
    wd_step=2.0,
    ws_step=1.0,
)
ax = er_out.plot_energy_ratios(overlay_frequency=True)
ax[0].set_title("Downstream Turbine: SMV5 (004)")

The FLORIS predictions agree reasonably well with the SCADA-based energy ratios, although FLORIS predicts slightly lower wake losses overall. Note that the SCADA-based energy ratios exceed 1 at the edge of the wake between 220 and 235 degrees, whereas FLORIS predicts energy ratios of 1. The higher SCADA-based energy ratios could indicate a speed up region at the edge of the wake where SMV5 outperforms the reference turbines.  

## Compare the energy ratio gains from SCADA and FLORIS

Lastly, we'll compare the changes in the energy ratios from wake steering based on the SCADA data with the equivalent FLORIS predictions for the downstream turbine adn the two turbines combined.

In [ ]:
# Downstream only
er_out = er.compute_energy_ratio(
    er_in,
    test_turbines=[4],
    use_predefined_ref=True,
    use_predefined_wd=True,
    use_predefined_ws=True,
    wd_step=2.0,
    ws_step=1.0,
    uplift_pairs=[
        ("Baseline [SCADA]", "Controlled [SCADA]"),
        ("Baseline [FLORIS]", "Controlled [FLORIS]"),
    ],
    uplift_names=["Uplift [SCADA]", "Uplift [FLORIS]"],
    N=40,
)
er_uplift_colors = {"Uplift [SCADA]": "black", "Uplift [FLORIS]": "blue", "weight": "red"}

ax = er_out.plot_uplift(color_dict=er_uplift_colors, overlay_frequency=True)
ax[0].set_title("Downstream Turbine: SMV5 (004)")

In [ ]:
# Combined
er_out = er.compute_energy_ratio(
    er_in,
    test_turbines=[4, 5],
    use_predefined_ref=True,
    use_predefined_wd=True,
    use_predefined_ws=True,
    wd_step=2.0,
    ws_step=1.0,
    uplift_pairs=[
        ("Baseline [SCADA]", "Controlled [SCADA]"),
        ("Baseline [FLORIS]", "Controlled [FLORIS]"),
    ],
    uplift_names=["Uplift [SCADA]", "Uplift [FLORIS]"],
    N=40,
)
ax = er_out.plot_uplift(color_dict=er_uplift_colors, overlay_frequency=True)
ax[0].set_title("Upstream and Downstream Turbines Combined: SMV5+SMV6 (004+005)")

In general, although the SCADA-based energy ratios are much noisier than the FLORIS predictions, the observed energy gains are relatively close to the predicted gains.

## Compare the total uplift amounts above 4m/s

In [ ]:
# Downstream only
total_uplift_result = tup.compute_total_uplift(
    er_in,
    test_turbines=[4, 5],
    use_predefined_ref=True,
    use_predefined_wd=True,
    use_predefined_ws=True,
    wd_step=2.0,
    wd_min=195.0,  # As in paper
    wd_max=240.0,  # As in paper
    ws_step=1.0,
    ws_min=4.0,
    uplift_pairs=[
        ("Baseline [SCADA]", "Controlled [SCADA]"),
        ("Baseline [FLORIS]", "Controlled [FLORIS]"),
    ],
    uplift_names=["Uplift [SCADA]", "Uplift [FLORIS]"],
    N=100,
    percentiles=(10, 90),  # Use P10 and P90
)


print(
    f"Percent increase in total energy production for combined turbines: "
    f"{total_uplift_result['Uplift [SCADA]']['energy_uplift_ctr_pc']:.3f}% (SCADA)"
)
print(
    f"Percent increase in total energy production for combined turbines: "
    f"{total_uplift_result['Uplift [FLORIS]']['energy_uplift_ctr_pc']:.3f}% (FLORIS)"
)
print(" ")
print("Full contents of dictionary including P10 and P90 values...")
total_uplift_result